In [250]:
import os
import pandas as pd
import nltk
import spacy
import nltk.data
from nltk.corpus import stopwords
from nltk.tokenize import line_tokenize
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.chunk import tree2conlltags
import re
from sklearn.model_selection import train_test_split
from nltk.cluster import KMeansClusterer
import gensim 
from gensim.models import Word2Vec
  
from nltk.cluster import KMeansClusterer
import nltk
import numpy as np 
  
from sklearn import cluster
from sklearn import metrics

In [467]:
#Set working directory
main_dir ='/Users/cxue200/Desktop/Amazon_transcript/'
os.chdir(main_dir)
## Read in all txt files into one list

location = os.getcwd() # get present working directory location here

df_list=[]

files=[]

for file in os.listdir(location):
    if file.endswith(".txt"):
        file_name=file.replace('.txt','')
        df = nltk.data.load(file)
        if (re.search("thank you for",df.lower())) or (re.search("my name",df.lower())) or (re.search("i help",df.lower())):
            #df_list.append(df)
            files.append(file_name)
        df_list.append(df)


In [345]:
#sales_chat=set()

#for i in range(len(df_list)):
#    split_df=df_list[i][1].split('.')
#    for sen in split_df:
#        if (re.search("thank you for",sen.lower())) or (re.search("my name",sen.lower())) or (re.search("i help",sen.lower())):
#            sales_chat.add(df_list[i][0])
#non_sales_chat=set(files)-sales_chat

In [468]:
# Dictionary for phrases transformation


dic={'home security system':'homesecuritysystem',\
     'security system':'securitysystem',\
     'home security':'homesecurity',
     'set top box':'settopbox',\
     'xfinity mobile':'xfinitymobile',\
     'infinity mobile':'xfinitymobile',\
     'land line':'landline',\
     'home phone':'homephone',\
     'hot spot':'hotspot',\
     'home phones':'homephone',\
     'quad play':'quadplay',\
     'double play':'doubleplay',\
     'triple play':'tripleplay',\
     'single play':'singleplay',\
     'performance pro':'performancepro',\
     'performance plus':'performanceplus',\
     'performance starter':'performancestarter',\
     'amazon prime': 'amazonprime',\
     'sling tv':'slingtv',\
     'direct tv':'directtv',\
     'high definition':'highdefinition',\
     'megabyte':'megabytes',\
     'megabits':'megabytes',\
     'premium channel':'premiumchannel',\
     'show time':'showtime',\
     'mobile number':'mobilenumber',\
     'mobile phone':'mobilephone',\
    }
    


# Function to replace keywords in text regardless of case
def replace_all(text, dic):
    for i, j in dic.items():
        src_str  = re.compile(i, re.IGNORECASE)
        text = src_str.sub(j,text)
    return text

# Transform and Tokenize each chat 
token_chat=[]
for chat in df_list:
    chat=replace_all(chat, dic) # convert phrases into one word format according to preset dictionary 
    token_txt = line_tokenize(chat)
    token_word = [nltk.word_tokenize(txt) for txt in token_txt]
    token_chat=token_chat+(token_word)

In [447]:
# Exclude hyphen from punctuation removal list
import re
import string

premove = string.punctuation
premove = premove.replace("-", "") # don't remove hyphens


#Specify a list of stop words to remove
more_stops=['I','say','hello','na','gon','get','uh','right','guy','still','come','say','someth','mean','um','oh','yes','bye','great','yeah','no','sorry','ok','okay','thank','like','alright','know']
stop_words = set(nltk.corpus.stopwords.words('english')+more_stops)

# Create a vector for clean chat.
chat_clean = [None] * len(token_chat)

# Create a vector of word counts for each clean chat.
word_counts_per_chat = [None] * len(token_chat)

# Process words in all chats.
for i in range(len(token_chat)):
    # 1. Convert to lower case.
    chat_clean[i] = [chat.lower() for chat in token_chat[i]]
    
    # 2. Remove stopwords.
    chat_clean[i] = [word for word in chat_clean[i] if not word in stop_words]
    
    # 3. Remove punctuation and any non-alphabetical characters.
    chat_clean[i] = [word for word in chat_clean[i] if word not in premove]
    
    # 4. Stem words.
    #chat_clean[i] = [PorterStemmer().stem(word) for word in chat_clean[i]]
    
    # Add POS and Name entity to words
    chat_clean[i]=tree2conlltags(ne_chunk(pos_tag((chat_clean[i]))))
    
    # Record the word count per chat.
    word_counts_per_chat[i] = len(chat_clean[i])

In [469]:
# Add chat id to each row as id 
new_tpl=[]
for i,fname in zip(range(len(chat_clean)),os.listdir(location)):
    if file.endswith(".txt"):
        #print(i,' ',fname)
        fname=fname.replace('.txt','')
        for tpl in chat_clean[i]:
            tpl_update=(fname,)+tpl
            new_tpl.append(tpl_update)
        
# Convert tuples to pandas dataframe
import numpy as np
df=pd.DataFrame(new_tpl, columns=['Chat_Num','Word','POS','Entity']).drop(['Entity','POS'], axis=1) 


In [472]:
# Product related keywords
hsd=['speed','internet','megabyte','megabytes','megabits','modem','gigabytes','signal','wifi','network','router']
video=['channel','channels','set-top-box','dvr','box','boxes']
cdv=['landline','homephone']
xm=['mobile','mobilephone','lines','hotspot','xfinity-mobile','hotspots']
xh=['securitysystem','homesecuritysystem','homesecurity','camera','cameras']

# Other Categories of keywords
stream=['streaming','stream']
ott=['netflix','hulu','roku']
hsd_tier=['blast','performance','performance-pro','performanceplus','performancestarter','gigabit']
premium_channel=['premiumchannel','hbo','espn','showtime','cinemax','movie channels']
vid_tier=['latino','dvr','hd','highdefinition']
#packages=['quadplay','doubleplay','tripleplay','singleplay']
tp=['tripleplay','triple']
dp=['doubleplay','double']
qp=['quadplay','quad']
upg=['upgrade']


# Tag keywords based on word categoriess
df['prod_tag'] = df['Word'].apply(lambda x: 'HSD' if x.lower() in hsd \
                             else 'VID' if x.lower() in video \
                             else 'CDV' if x.lower() in cdv \
                             else 'XH' if x.lower() in xh \
                             else 'XM' if x.lower() in xm \
                             else 'TP' if x.lower() in tp \
                             else 'DP' if x.lower() in dp \
                             else 'QP' if x.lower() in qp else 'NA')


df['tier_tag'] = df['Word'].apply(lambda x: 'HSD_TIER' if x.lower() in hsd_tier \
                             else 'VID_TIER' if x.lower() in vid_tier \
                             else 'PREM_CHANNEL' if x.lower() in premium_channel else 'NA')


df['other_tag'] = df['Word'].apply(lambda x: 'OTT' if x.lower() in stream \
                             else 'STREAM' if x.lower() in ott
                             else 'UPGRADE' if x.lower in upg else 'NA')


In [474]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Calcultae TFIDF weight for only keywords in each chat

td_all=pd.DataFrame()
n=0

for i,fname in zip(range(len(df_list)),os.listdir(location)):
    fname=fname.replace('.txt','')
    n=i
    vec = CountVectorizer()
    X = vec.fit_transform(([df_list[i]]))
    
    # TF_IDF Transformation
    tf_idf = TfidfTransformer(norm="l2")
    tf_idf.fit(X)
    X_tf_idf = tf_idf.transform(X)
    
    exercise_DTM_tf_idf = pd.DataFrame(X_tf_idf.toarray(), columns = vec.get_feature_names())
    #print(exercise_DTM_tf_idf)

    allkeys=video+hsd+cdv+xh+xm+hsd_tier+vid_tier+premium_channel+tp+dp+qp+stream+ott
    
    
    def include_keys(dictionary, keys):
        """Filters a dict by only including certain keys."""
        key_set = set(keys) & set(dictionary.keys())
        return {key: dictionary[key] for key in key_set}
    
    def HeadDict(dict_x, n):
        # Focus on weight of words on the list only 
        tempdic=include_keys(dict_x,allkeys)

        # Get items from the dictionary and sort them by
        # value key in descending (i.e. reverse) order
        sorted_x = sorted(tempdic.items(),
                    reverse = True,
                    key = lambda kv: kv[1])

        # Convert sorted dictionary to a list.
        dict_x_list = list(sorted_x)

        # Return the first `n` values from the dictionary only.
        return(dict(dict_x_list[:n]))

    #exercise_DTM_tf_idf.sum(axis = 0).head()
    
    # Save series as a dictionary.
    corpus_freq_dist_tf_idf = exercise_DTM_tf_idf.sum(axis = 0).to_dict()
    td_dic=HeadDict(corpus_freq_dist_tf_idf,1000)

    if len(td_dic)==0:
        df_td=pd.DataFrame([['None', 0]],columns=['Word','Weight'])
        df_td['Chat_Num']=fname
        df_td=df_td[['Chat_Num', 'Word','Weight']]
    else:
        df_td=pd.DataFrame.from_dict(td_dic,orient='index').reset_index()
        df_td.columns=['Word','Weight']
        df_td['Chat_Num']=fname
        df_td=df_td[['Chat_Num', 'Word','Weight']]
    
    df=df_td
    # Tag keywords with theme names
    df['prod_tag'] = df['Word'].apply(lambda x: 'HSD' if x.lower() in hsd \
                                 else 'VID' if x.lower() in video \
                                 else 'CDV' if x.lower() in cdv \
                                 else 'XH' if x.lower() in xh \
                                 else 'XM' if x.lower() in xm \
                                 else 'TP' if x.lower() in tp \
                                 else 'DP' if x.lower() in dp \
                                 else 'QP' if x.lower() in qp else 'NA')
    
    
    df['tier_tag'] = df['Word'].apply(lambda x: 'HSD_TIER' if x.lower() in hsd_tier \
                                 else 'VID_TIER' if x.lower() in vid_tier \
                                 else 'PREM_CHANNEL' if x.lower() in premium_channel else 'NA')
    
    
    df['other_tag'] = df['Word'].apply(lambda x: 'OTT' if x.lower() in stream \
                                 else 'STREAM' if x.lower() in ott 
                                 else 'UPGRADE' if x.lower in upg else 'NA')
    
    #Join all the chats 
    td_all = pd.concat([td_all,df],axis=0)
    

In [493]:
tier_agg_td=td_all.groupby(['Chat_Num','tier_tag'])[['Weight']].sum()\
.sort_values(["Chat_Num","Weight"], inplace=False,ascending=[True,False])\
.reset_index()

prod_agg_td=td_all.groupby(['Chat_Num','prod_tag'])[['Weight']].sum()\
.sort_values(["Chat_Num","Weight"], inplace=False,ascending=[True,False])\
.reset_index()

other_agg_td=td_all.groupby(['Chat_Num','other_tag'])[['Weight']].sum()\
.sort_values(["Chat_Num","Weight"], inplace=False,ascending=[True,False])\
.reset_index()


#prod_agg_td=prod_agg_td[prod_agg_td.prod_tag!='NA']
prod_agg_td.columns = ['Chat_Num', 'Tag','Weight']
#tier_agg_td=tier_agg_td[tier_agg_td.tier_tag!='NA']
tier_agg_td.columns = ['Chat_Num', 'Tag','Weight']
#other_agg_td=other_agg_td[other_agg_td.other_tag!='NA']
other_agg_td.columns = ['Chat_Num', 'Tag','Weight']


td_final=pd.concat([prod_agg_td,tier_agg_td,other_agg_td],ignore_index=True).sort_values(by=['Chat_Num','Weight'],ascending=False)

In [500]:
tag_cnt_df=td_final.groupby(['Chat_Num']).Chat_Num.agg('count').to_frame('Tag_Count').reset_index()
td_final.loc[td_final.Weight<0.006, 'Tag'] = 'None'
#td_final.loc[td_final.Weight<0.006, 'Weight'] = 0
td_final=td_final[td_final.Tag!='NA']
td_final=td_final[td_final.Tag!='None']


#Formatting output
td_final_group=td_final.drop_duplicates().groupby('Chat_Num')['Tag'].apply(' '.join).reset_index()
td_final_group=pd.merge(td_final_group, tag_cnt_df, on='Chat_Num')
td_final_group['Tag']=np.where(~td_final_group['Chat_Num'].isin(files),'NON CUSTOMER CALL',
                               np.where(td_final_group['Tag_Count']<3,'NOT ENOUGH EVIDENCE',td_final_group.Tag))

In [501]:
td_final_group

,Chat_Num,Tag,Tag_Count
0,10001377861551369299,HSD OTT XM HSD_TIER VID STREAM,6
1,10022302631550499038,VID HSD OTT PREM_CHANNEL,5
2,10023088811549666899,HSD VID OTT,3
3,10023104301550533812,HSD VID XM PREM_CHANNEL OTT XH VID_TIER,10
4,10024064341550948008,HSD HSD_TIER VID XM VID_TIER OTT,6
5,10024116651550778579,VID VID_TIER HSD,4
6,10024338221551288668,VID HSD TP VID_TIER PREM_CHANNEL OTT,8
7,10024490731550175745,VID HSD,3
8,10026076441549661295,HSD VID TP VID_TIER,5
9,10026077961551118639,HSD VID VID_TIER,3


In [443]:
for tag in ['VID','HSD','XH','XM','HSD_TIER','VID_TIER','PREM_CHANNEL','TP','DP','QP','STREAM','OTT','UPGRADE']:
    td_final_group[tag]=td_final_group['Tag'].apply(lambda x: 1 if tag in x else 0 )


In [444]:
td_final_group

,Chat_Num,Tag,Tag_Count,VID,HSD,XH,XM,HSD_TIER,VID_TIER,PREM_CHANNEL,TP,DP,QP,STREAM,OTT,UPGRADE
0,10001377861551369299,HSD OTT XM HSD_TIER VID STREAM,6,1,1,0,1,1,0,0,0,0,0,1,1,0
1,10022302631550499038,VID HSD OTT PREM_CHANNEL,4,1,1,0,0,0,0,1,0,0,0,0,1,0
2,10023088811549666899,HSD VID OTT,3,1,1,0,0,0,0,0,0,0,0,0,1,0
3,10023104301550533812,HSD VID XM PREM_CHANNEL OTT XH VID_TIER,7,1,1,1,1,0,1,1,0,0,0,0,1,0
4,10024064341550948008,HSD HSD_TIER VID XM VID_TIER OTT,6,1,1,0,1,1,1,0,0,0,0,0,1,0
5,10024116651550778579,VID VID_TIER HSD,3,1,1,0,0,0,1,0,0,0,0,0,0,0
6,10024338221551288668,VID HSD TP VID_TIER PREM_CHANNEL OTT,6,1,1,0,0,0,1,1,1,0,0,0,1,0
7,10024490731550175745,NOT ENOUGH EVIDENCE,2,1,0,0,0,0,0,0,0,0,0,0,0,0
8,10026076441549661295,HSD VID TP VID_TIER,4,1,1,0,0,0,1,0,1,0,0,0,0,0
9,10026077961551118639,HSD VID VID_TIER,3,1,1,0,0,0,1,0,0,0,0,0,0,0


# Word2Vec + Clustering 

In [36]:
# Create Skip Gram model 
model2 = gensim.models.Word2Vec(chat_clean, min_count = 1, size = 100, 
                                             window = 5, sg = 1) 

print("Cosine similarity between 'video' " + 
               "and 'internet' - CBOW : ", 
    model2.similarity('speed', 'internet')) 

Cosine similarity between 'video' and 'internet' - CBOW :  0.964127515522


In [65]:
model = gensim.models.Word2Vec(chat_clean, min_count = 1, size = 100, 
                                             window = 7, sg = 1) 
  
X = model[model.wv.vocab]  

# note with some version you would need use this (without wv) 
#  model[model.vocab] 
#print (model[model.wv.vocab])
 
#print (model.similarity('post', 'book'))
#print (model.most_similar(positive=['machine'], negative=[], topn=2))
  
  
import nltk
NUM_CLUSTERS=8
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=50)
assigned_clusters = kclusterer.cluster(X, assign_clusters=True)
#print (assigned_clusters)


words = list(model.wv.vocab)
for i, word in enumerate(words):  
    print (word + ":" + str(assigned_clusters[i]))
 


much:5
calling:11
comcast:11
chris:9
getting:2
technical:9
support:7
may:11
help:11
bill:1
normally:4
little:6
sales:7
people:2
call:5
two:1
hours:3
ago:4
mhm:5
totally:9
time:6
n't:6
talk:12
's:5
running:7
slow:8
three:13
weeks:7
something:2
looking:2
picks:0
said:2
'll:14
give:5
hundred:1
megabytess:3
plus:1
thirty:1
nine:1
twenty:1
dollars:1
month:1
would:3
work:6
minutes:14
couple:7
checked:8
'm:5
slowing:8
talked:8
spending:8
ten:1
called:2
got:5
automated:12
service:0
payment:2
paying:1
guess:6
painted:8
actually:4
slower:7
need:5
check:5
see:5
increase:1
internet:0
speed:3
core:8
rolls:10
sure:5
definitely:6
take:5
look:5
real:9
quick:12
last:13
four:1
digits:10
social:13
account:13
please:14
six:13
seven:13
awesome:7
thanks:12
moment:5
pull:12
let:5
boot:9
file:10
looks:4
extreme:3
black:8
blast:0
fifty:1
megabytes:3
speak:9
testing:10
someone:8
p:3
sixty:1
try:5
first:5
one:3
report:4
times:4
go:5
five:1
run:7
tests:7
girl:8
person:6
spoke:8
computers:8
never:6
went:4
watching

comm:7
attack:8
mutual:7
innocent:8
young:8
corporation:8
san:8
floor:8
shipped:7
become:7
focus:8
govern:8
typical:8
causes:4
sposa:8
animals:8
audiobook:8
promises:8
samples:8
airports:7
desk:8
comedic:7
amanda:8
heart:7
supplementation:8
intern:7
exist:8
internets:7
sister:7
dancing:7
rely:8
supermarket:8
traveling:7
players:7
continually:7
entitled:7
airport:7
insane:8
equally:7
commanders:8
francis:8
conolly:8
humidity:7
canisters:8
pocket:7
slam:8
phoenix:8
analyst:8
english:7
comments:8
reporter:8
terrorist:9
latina:7
sporting:7
parkland:7
economically:8
western:8
north:10
presidency:8
sunday:8
latinos:8
minus:7
teller:8
older:8
consumo:7
future:7
laughing:8
apart:8
rising:8
saluting:8
upper:8
cindy:8
muslim:8
starting:4
amazed:8
protesters:7
porn:7
california:8
cab:8
lay:8
others:8
incorporated:8
idea:8
suspected:8
replicate:9
suitcase:8
underestimate:8
americans:8
locate:8
concern:8
indian:7
stella:8
commander:8
pursuing:7
occurrence:8
generate:8
images:8
seem:7
unanimously:8


In [69]:
df_cluster = pd.DataFrame(list(zip(words, assigned_clusters)), columns =['Word', 'Cluster']) 